In [4]:
_input_df = "select (Version.[Version Name]*Product.[Product]*Time.FiscalWeek*SalesAccount.[Account]*Location.[Location]*{Measure.[DPSellOutUnitsActuals],Measure.[Mean Pricing Save PCT],Measure.[Placement Count],Measure.[Promotion Count],Measure.[DPSellOutPrice]});"
_predict_df = "select (Version.[Version Name]*Product.[Product]*Time.FiscalWeek*SalesAccount.[Account]*Location.[Location]*{Measure.[DPSellOutUnitsActuals],Measure.[Mean Pricing Save PCT],Measure.[Placement Count],Measure.[Promotion Count],Measure.[DPSellOutPrice]});"

from o9_common_utils.O9DataLake import O9DataLake, ResourceType, DataSource,PluginSetting

predict_df = O9DataLake.register("predict_df",data_source = DataSource.LS, entity_type = ResourceType.IBPL, query = _predict_df,plugin_setting = PluginSetting.Inputs)
input_df = O9DataLake.register("input_df",data_source = DataSource.LS, entity_type = ResourceType.IBPL, query = _input_df,plugin_setting = PluginSetting.Inputs)

# register slice dimension
O9DataLake.register("Product.[Product]", data_source = DataSource.LS, entity_type = ResourceType.IBPL, plugin_setting = PluginSetting.SliceDimension)

# register outputs
O9DataLake.register("out_df",data_source = DataSource.LS,entity_type = ResourceType.IBPL, plugin_setting = PluginSetting.Outputs)

In [5]:
# fetching inputs
predict_df = O9DataLake.get('predict_df')
input_df = O9DataLake.get('input_df')

In [6]:
"""
exec plugin instance [CsvMGIP1_CsvMGIP2_CsvMGPrimDf_CsvMGOP1_Slice] for measures {[DPSellOutUnitsActuals]} using scope ([Product].[Product].Filter(#.Name in {[196426],[208821]}) * [Time].[Day] * [Version].[Version Name].[CurrentWorkingView]);

"""
import logging
from sklearn import tree
import logging
logger = logging.getLogger('o9_logger')
logger.info("######################################")
logger.error(f'Ritu Predict df {predict_df.shape}')
logger.error(f'Ritu input df {input_df.shape}')
logger.error(f'Ritu Predict df count {predict_df.count()}')
logger.error(f'Ritu input df count {input_df.count()}')
logger.error(f'Ritu Predict df columns {predict_df.columns}')
logger.error(f'Ritu input df columns {input_df.columns}')
logger.info("######################################")

out_df = None

if predict_df is not None and len(predict_df.index) > 0:
    x = input_df[['Mean Pricing Save PCT', 'Placement Count', 'Promotion Count', 'DPSellOutPrice']]
    y = input_df[['DPSellOutUnitsActuals']]
    x = x.fillna(0).values
    y = y.fillna(0).values

    clf = tree.DecisionTreeRegressor()
    clf = clf.fit(x, y)

    predict = predict_df[['Mean Pricing Save PCT', 'Placement Count', 'Promotion Count', 'DPSellOutPrice']].fillna(0).values
    # predict_keys = predict_df[['Time.[FiscalWeek]', 'Product.[Product]', 'Version.[Version Name]', 'SalesAccount.[Account]',                               'Location.[Location]']]
    # predict_keys["Product.[Product]"] = pd.to_numeric(predict_keys["Product.[Product]"], downcast='integer')
    predict_values = clf.predict(predict)
    predict_df["DPSellOutUnitsFcst"] = predict_values
    #out_df = pd.concat([predict_keys, out_df_part], axis=1)
    out_df=predict_df[['Time.[FiscalWeek]', 'Product.[Product]', 'Version.[Version Name]', 'SalesAccount.[Account]','Location.[Location]','DPSellOutUnitsFcst']]
    logger.info(out_df)

else:
    out_df = None


2023-11-14 14:03:47,839 - o9_logger - INFO  - ######################################
2023-11-14 14:03:47,841 - o9_logger - ERROR  - Ritu Predict df (200000, 10)
2023-11-14 14:03:47,842 - o9_logger - ERROR  - Ritu input df (200000, 10)
2023-11-14 14:03:47,933 - o9_logger - ERROR  - Ritu Predict df count Version.[Version Name]    200000
Product.[Product]         200000
Time.[FiscalWeek]         200000
SalesAccount.[Account]    200000
Location.[Location]       200000
DPSellOutUnitsActuals     100910
Mean Pricing Save PCT      12198
Placement Count           167164
Promotion Count            12198
DPSellOutPrice             99692
dtype: int64
2023-11-14 14:03:47,993 - o9_logger - ERROR  - Ritu input df count Version.[Version Name]    200000
Product.[Product]         200000
Time.[FiscalWeek]         200000
SalesAccount.[Account]    200000
Location.[Location]       200000
DPSellOutUnitsActuals     100910
Mean Pricing Save PCT      12198
Placement Count           167164
Promotion Count       